In [1]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image
import os

In [2]:
art = pd.read_csv('../Data/Artist/all_select.csv')
art.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [3]:
idx = np.random.randint(0, len(art), size=50)
idx

array([  5106,  34132,  70935,  42209,  94507,  48972,  39472,  29295,
        29474,   1269,  87323,  27498,  25524,  31277,  28078,  85780,
        43854,  32984,  35686,  80678,  60485,  63578,  81714,  11399,
        80290,  83355,  41491,  32661,   5629, 100777,  98441,   4582,
        15595,  40128,  22997,  97187,  34364,  93595,  94944,   8282,
        23232,  53623,  88581,  39378,  49168,  77719,  96078,  14181,
        81478,  15007])

In [4]:
file_list = art.iloc[idx]['new_filename'].to_list()
file_list[:5]


# 랜덤으로 선택한 그림정보 생성 - 스케치 파일 이거나 가로세로비가 너무 클 경우 학습정보에서 배제
current_selet = art.iloc[idx].reset_index(drop=False)
current_selet.to_csv('../Data/Artist/current_select.csv', index=False)

---
# -15 ~ 15도 까지 0.5도씩 회전하여 영상 생성

In [5]:
def imageGenerate(src, max_height, angle):
    
    max_width = np.uint(max_height * 1.5)

    height, width = src.shape[0:2]
    ratio = height / max_height
    
    # 입력 이미지의 높이가 설정한 이미지의 최대 높이보다 크면 리사이즈 수행
    if height > max_height :
        resize_height = max_height
        resize_width = np.uint(width // ratio)
        img = cv2.resize(src, (resize_width, resize_height), cv2.INTER_AREA)
        dy = 0
    
    # 입력 이미지의 높이가 설정한 이미지의 최대 높이보다 작으면 리사이즈 수행하지 않음
    else : 
        resize_height = height
        resize_width = width
        dy = (max_height - resize_height) // 2

    dx = (max_width - resize_width) // 2
    mtrx = np.float32([[1,0,dx],[0,1,dy]])

    # 입력 이미지를 규격 프레임의 중심에 넣는다
    img = cv2.warpAffine(img, mtrx, (max_width, max_height))

    # 이미지 회전
    mtrx_rot = cv2.getRotationMatrix2D((max_width/2, max_height/2), angle, 0.9)
    return cv2.warpAffine(img, mtrx_rot, (max_width,max_height))  
     

In [6]:
# 각도범위 -15 ~ 15도 이며 0.5도씩 증가
angles = (np.arange(-15, 15, 0.5))
angles

array([-15. , -14.5, -14. , -13.5, -13. , -12.5, -12. , -11.5, -11. ,
       -10.5, -10. ,  -9.5,  -9. ,  -8.5,  -8. ,  -7.5,  -7. ,  -6.5,
        -6. ,  -5.5,  -5. ,  -4.5,  -4. ,  -3.5,  -3. ,  -2.5,  -2. ,
        -1.5,  -1. ,  -0.5,   0. ,   0.5,   1. ,   1.5,   2. ,   2.5,
         3. ,   3.5,   4. ,   4.5,   5. ,   5.5,   6. ,   6.5,   7. ,
         7.5,   8. ,   8.5,   9. ,   9.5,  10. ,  10.5,  11. ,  11.5,
        12. ,  12.5,  13. ,  13.5,  14. ,  14.5])

In [7]:
read_path = '../Data/Artist/original/org_select/'
save_path = '../Data/Artist/train/'

max_height = 400

for file in file_list:
    try :
        file_path = read_path + file
        img = cv2.imread(file_path, cv2.IMREAD_COLOR)
        idx = 0

        (height, width) = img.shape[0:2]
        btm = np.mean(img)
        
        if (width / height) < 1.5 and btm < 185 : # 가로세로비 1.5 이하이며 밝기 평균이 185(스케치 제외) 미만인 이미지만 선택
            for angle in angles : 
                file_name = save_path + file.replace('.jpg', '') + '_%02d'%idx + '.jpg'
                img2 = imageGenerate(img, max_height, angle)
                cv2.imwrite(file_name, img2)
                idx += 1  
    except :
        pass
